In [50]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sother', 
    'sage',
    'sempl', 
    'sincome',
    'spop', 
    'sbehavior',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    'omajother', 
    'osplit', 
    
    # Encounter characteristics
    'daytime',
    'inctype_lin', 
    'year'
]

# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',
    'sincome', 
    'sempl', 
    'spop', 
    'sbehavior',

    # Officer vars (white is reference)
    'omajblack',
    'omajhisp',
    'omajother', 

    # Encounter vars
    'daytime'
]

In [52]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [53]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))
y.shape

Shape x: (3799, 7)
Rank x: 7


(3799,)

In [54]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

LPM results
Dependent variable: anyuseofforce_coded

R2 = 0.005
sigma2 = nan


,b_hat,se,t
sblack,0.0054,0.0043,1.2496
shisp,0.0126,0.0063,1.9958
smale,0.0069,0.0022,3.0897
sage,-0.0000,0.0000,-0.3901
omajblack,-0.0051,0.0017,-3.0929
omajhisp,0.0038,0.0107,0.3515
daytime,-0.0009,0.0022,-0.4098


In [55]:
theta0 = probit.starting_values(y, x)
theta0.ndim==1

True

In [61]:
ll = probit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll), -1.0411283428047824)

np.False_

In [62]:
probit_results = est.estimate(probit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.092128
         Iterations: 1
         Function evaluations: 16
         Gradient evaluations: 2


LinAlgError: Singular matrix

In [43]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

NameError: name 'probit_results' is not defined

In [57]:
theta0 = logit.starting_values(y, x)
theta0 

array([ 2.15217100e-02,  5.04941424e-02,  2.76702512e-02, -6.22245366e-05,
       -2.04992618e-02,  1.50956695e-02, -3.68034836e-03])

In [58]:
ll = logit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll),-0.9974267061091704)

np.False_

In [59]:
logit_results = est.estimate(logit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.032884
         Iterations: 85
         Function evaluations: 816
         Gradient evaluations: 102


In [60]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 85 iter. (816 func. evals.). Final criterion:  0.03288.
Logit, y = anyuseofforce_coded


,theta,se,t
sblack,0.2876,0.7204,0.3993
shisp,0.6274,0.5328,1.1776
smale,-0.0290,0.6603,-0.0439
sage,-0.1602,0.0168,-9.5142
omajblack,-10.6629,156256.9018,-0.0001
omajhisp,0.2650,1.3061,0.2029
daytime,-0.9547,0.5760,-1.6576
